# Answers to coding questions in deep learning
## By Peeter Niidas

In [56]:
import pandas as pd
import numpy as np
import os
import sys
import glob
import matplotlib.pyplot as plt
#%tensorflow_version 1.x

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [57]:
# Get filenames and load data

def read_rename_data (dirr):
    path = dirr
    i = 0
    for filename in glob.iglob(path + '**/*.json', recursive=True):
        if i < 100:
            print(filename)
            i += 1
            df = pd.read_json(filename)
    return df

In [58]:
df_true = read_rename_data('./data/pheme-rnr-dataset/sydneysiege/non-rumours/')

./data/pheme-rnr-dataset/sydneysiege/non-rumours/544282393801007104/source-tweet/544282393801007104.json
./data/pheme-rnr-dataset/sydneysiege/non-rumours/544282393801007104/reactions/544285550824669184.json
./data/pheme-rnr-dataset/sydneysiege/non-rumours/544282393801007104/reactions/544287961559203840.json
./data/pheme-rnr-dataset/sydneysiege/non-rumours/544282393801007104/reactions/544284195074629632.json
./data/pheme-rnr-dataset/sydneysiege/non-rumours/544282393801007104/reactions/544540145803071488.json
./data/pheme-rnr-dataset/sydneysiege/non-rumours/544282393801007104/reactions/544312169597517825.json
./data/pheme-rnr-dataset/sydneysiege/non-rumours/544282393801007104/reactions/544372236971286528.json
./data/pheme-rnr-dataset/sydneysiege/non-rumours/544282393801007104/reactions/544289020071923712.json
./data/pheme-rnr-dataset/sydneysiege/non-rumours/544282393801007104/reactions/544325510033969152.json
./data/pheme-rnr-dataset/sydneysiege/non-rumours/544282393801007104/reactions/5

In [59]:
df_rumors = read_rename_data('./data/pheme-rnr-dataset/sydneysiege/rumours/')

./data/pheme-rnr-dataset/sydneysiege/rumours/544287499397242880/source-tweet/544287499397242880.json
./data/pheme-rnr-dataset/sydneysiege/rumours/544287499397242880/reactions/544291686738710528.json
./data/pheme-rnr-dataset/sydneysiege/rumours/544287499397242880/reactions/544289337517436928.json
./data/pheme-rnr-dataset/sydneysiege/rumours/544287499397242880/reactions/544301824791306240.json
./data/pheme-rnr-dataset/sydneysiege/rumours/544287499397242880/reactions/544291075553107971.json
./data/pheme-rnr-dataset/sydneysiege/rumours/544287499397242880/reactions/544302633553375232.json
./data/pheme-rnr-dataset/sydneysiege/rumours/544512691948498944/source-tweet/544512691948498944.json
./data/pheme-rnr-dataset/sydneysiege/rumours/544512691948498944/reactions/544513096975253507.json
./data/pheme-rnr-dataset/sydneysiege/rumours/544512691948498944/reactions/544512909712171009.json
./data/pheme-rnr-dataset/sydneysiege/rumours/544512691948498944/reactions/544513097550274561.json
./data/pheme-r

In [60]:
df_true.head()

,contributors,truncated,text,in_reply_to_status_id,id,favorite_count,source,retweeted,coordinates,entities,...,retweet_count,in_reply_to_user_id,favorited,user,geo,in_reply_to_user_id_str,lang,created_at,in_reply_to_status_id_str,place
symbols,NaN,False,@RT_com ISLAM is being hijacked by ISIS. #ill...,544482664515121153,544482768328351744,0,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,[],...,0,64643056,False,NaN,NaN,64643056,en,2014-12-15 13:23:02+00:00,544482664515121152,NaN
user_mentions,NaN,False,@RT_com ISLAM is being hijacked by ISIS. #ill...,544482664515121153,544482768328351744,0,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,"[{'id': 64643056, 'indices': [0, 7], 'id_str':...",...,0,64643056,False,NaN,NaN,64643056,en,2014-12-15 13:23:02+00:00,544482664515121152,NaN
hashtags,NaN,False,@RT_com ISLAM is being hijacked by ISIS. #ill...,544482664515121153,544482768328351744,0,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,"[{'indices': [42, 57], 'text': 'illridewithyou'}]",...,0,64643056,False,NaN,NaN,64643056,en,2014-12-15 13:23:02+00:00,544482664515121152,NaN
urls,NaN,False,@RT_com ISLAM is being hijacked by ISIS. #ill...,544482664515121153,544482768328351744,0,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,[],...,0,64643056,False,NaN,NaN,64643056,en,2014-12-15 13:23:02+00:00,544482664515121152,NaN
follow_request_sent,NaN,False,@RT_com ISLAM is being hijacked by ISIS. #ill...,544482664515121153,544482768328351744,0,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,...,0,64643056,False,False,NaN,64643056,en,2014-12-15 13:23:02+00:00,544482664515121152,NaN


In [61]:
df_rumors.head()

,contributors,truncated,text,in_reply_to_status_id,id,favorite_count,source,retweeted,coordinates,entities,...,retweet_count,in_reply_to_user_id,favorited,user,geo,in_reply_to_user_id_str,lang,created_at,in_reply_to_status_id_str,place
symbols,NaN,False,@JustNathaIy There's lots more on both sides.....,544479493436354560,544498376537280512,0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,NaN,[],...,0,125562479,False,NaN,NaN,125562479,en,2014-12-15 14:25:03+00:00,544479493436354560,NaN
user_mentions,NaN,False,@JustNathaIy There's lots more on both sides.....,544479493436354560,544498376537280512,0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,NaN,"[{'id': 125562479, 'indices': [0, 12], 'id_str...",...,0,125562479,False,NaN,NaN,125562479,en,2014-12-15 14:25:03+00:00,544479493436354560,NaN
hashtags,NaN,False,@JustNathaIy There's lots more on both sides.....,544479493436354560,544498376537280512,0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,NaN,[],...,0,125562479,False,NaN,NaN,125562479,en,2014-12-15 14:25:03+00:00,544479493436354560,NaN
urls,NaN,False,@JustNathaIy There's lots more on both sides.....,544479493436354560,544498376537280512,0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,NaN,[],...,0,125562479,False,NaN,NaN,125562479,en,2014-12-15 14:25:03+00:00,544479493436354560,NaN
follow_request_sent,NaN,False,@JustNathaIy There's lots more on both sides.....,544479493436354560,544498376537280512,0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,NaN,NaN,...,0,125562479,False,False,NaN,125562479,en,2014-12-15 14:25:03+00:00,544479493436354560,NaN


In [62]:
df_true.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, symbols to is_translator
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   contributors               0 non-null      float64            
 1   truncated                  44 non-null     bool               
 2   text                       44 non-null     object             
 3   in_reply_to_status_id      44 non-null     int64              
 4   id                         44 non-null     int64              
 5   favorite_count             44 non-null     int64              
 6   source                     44 non-null     object             
 7   retweeted                  44 non-null     bool               
 8   coordinates                0 non-null      float64            
 9   entities                   4 non-null      object             
 10  in_reply_to_screen_name    44 non-null     object             
 